# Model Context Protocol (MCP) with LangChain 1.0

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain/blob/main/notebooks/5.1-mcp-external-tools.ipynb)

**Note**: MCP may not work in Google Colab due to subprocess and stdio transport requirements. For best results, run this notebook locally.

## Overview

This notebook introduces the **Model Context Protocol (MCP)**, an open standard that revolutionizes how AI applications interact with external tools and data sources.

### What is MCP?

Model Context Protocol (MCP) is an open standard created by Anthropic (and now donated to the Agentic AI Foundation under the Linux Foundation in December 2025) that:

- **Standardizes** how AI applications connect to external tools and data sources
- **Simplifies** integration by providing a universal interface (think "USB-C for AI")
- **Enables reusability** - write an MCP server once, use it across any MCP-compatible application
- **Supports ecosystem growth** - hundreds of pre-built MCP servers are available for popular services

### Key Benefits

1. **No more custom integrations** - Use pre-built MCP servers instead of writing custom code for each API
2. **Multi-server management** - Connect to multiple MCP servers simultaneously
3. **Framework adoption** - Supported by LangChain, CrewAI, Pydantic.AI, and more
4. **Industry backing** - Adopted by OpenAI, Anthropic, Google, Microsoft, AWS, and others

### When to Use MCP?

**Use MCP when:**
- You need to integrate with services that have existing MCP servers (GitHub, Slack, databases, filesystems, etc.)
- You want to reuse tools across multiple projects or frameworks
- You're building production applications that need standardized tool interfaces

**Use custom tools when:**
- You have simple, project-specific tool requirements
- You're prototyping and need quick iteration
- No suitable MCP server exists for your use case

## Setup and Installation

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "langchain>=1.0.0",
#     "langchain-core>=1.0.0",
#     "langchain-openai",
#     "langchain-mcp-adapters>=0.1.0",
#     "langgraph>=1.0.0",
# ]
# ///

# Install required packages
# %pip install -qU langchain>=1.0.0
# %pip install -qU langchain-core>=1.0.0
# %pip install -qU langchain-openai
# %pip install -qU langchain-mcp-adapters>=0.1.0
# %pip install -qU langgraph>=1.0.0

### API Key Configuration

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

# Optional: Enable LangSmith tracing
# _set_env("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Understanding MCP Architecture

MCP follows a client-server architecture:

```
┌─────────────────┐
│  Your LangChain │
│   Application   │
└────────┬────────┘
         │
         │ uses
         ↓
┌─────────────────┐
│  MCP Client     │ ← MultiServerMCPClient
│  (langchain-mcp)│
└────────┬────────┘
         │
         │ connects to (stdio/HTTP)
         │
    ┌────┴────┬─────────┬──────────┐
    ↓         ↓         ↓          ↓
┌────────┐ ┌────────┐ ┌─────┐ ┌──────┐
│  File  │ │  Time  │ │ Git │ │Slack │
│ System │ │ Server │ │ MCP │ │ MCP  │
│  MCP   │ │  MCP   │ │     │ │      │
└────────┘ └────────┘ └─────┘ └──────┘
   MCP Servers (provide tools)
```

**Key Concepts:**
- **MCP Client** - Connects to multiple MCP servers and loads their tools
- **MCP Servers** - Provide tools, resources, and prompts through standardized protocol
- **Transport** - Communication method (stdio for local processes, HTTP for remote servers)
- **Tools** - Functions that can be called by LLMs (automatically converted to LangChain format)

## Example 1: Basic MCP Client Setup

Let's start with a conceptual example showing how to configure an MCP client. This demonstrates the pattern even if MCP servers aren't available in your environment.

In [ ]:
# CONCEPTUAL EXAMPLE - Shows the MCP client configuration pattern

# This is how you would configure MultiServerMCPClient with MCP servers:

mcp_config_example = """
from langchain_mcp_adapters.client import MultiServerMCPClient

# Configure MCP client with stdio transport (for local processes)
client = MultiServerMCPClient(
    servers={
        "filesystem": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-filesystem", "/tmp"],
        },
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-time"],
        },
    }
)

# MCP requires async context
async with client:
    # Get available tools from all connected servers
    tools = await client.get_tools()
    print(f"Loaded {len(tools)} tools from MCP servers")
    
    # Each tool includes:
    # - name: The tool identifier
    # - description: What the tool does
    # - input_schema: Parameters the tool accepts
    
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")
"""

print("MCP Client Configuration Pattern:")
print(mcp_config_example)

### Key Configuration Elements

1. **Transport Type**: How to communicate with the MCP server
   - `stdio` - For local processes (most common)
   - `http` - For remote servers (streamable HTTP in protocol 2025-03-26+)

2. **Command & Args**: How to start the MCP server process
   - Often uses `npx` to run Node.js-based MCP servers
   - Can use Python, Go, or any executable that implements MCP protocol

3. **Server Naming**: Choose descriptive names for each server (e.g., "filesystem", "time")
   - Helps identify which server provides which tools

## Example 2: Inspecting MCP Tool Schemas

MCP tools come with rich schemas that LangChain can automatically convert to its tool format.

In [ ]:
# CONCEPTUAL EXAMPLE - Demonstrates tool inspection pattern

tool_inspection_example = """
async with client:
    tools = await client.get_tools()
    
    # Inspect the first tool
    if tools:
        tool = tools[0]
        print(f"Tool Name: {tool.name}")
        print(f"Description: {tool.description}")
        print(f"Input Schema: {tool.input_schema}")
        
        # Example output for filesystem 'read_file' tool:
        # Tool Name: read_file
        # Description: Read the complete contents of a file from the file system
        # Input Schema: {
        #     "type": "object",
        #     "properties": {
        #         "path": {"type": "string", "description": "Path to file"}
        #     },
        #     "required": ["path"]
        # }
        
    # Get tools from a specific server
    filesystem_tools = [t for t in tools if 'filesystem' in t.name or 'file' in t.name]
    print(f"\nFilesystem-related tools: {len(filesystem_tools)}")
"""

print("Tool Inspection Pattern:")
print(tool_inspection_example)

## Example 3: Using MCP Tools with LangChain Agents

The real power of MCP comes from using these tools with LangChain agents.

In [ ]:
# CONCEPTUAL EXAMPLE - Shows MCP + LangChain agent integration

agent_integration_example = """
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

# Configure MCP client
client = MultiServerMCPClient(
    servers={
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-time"],
        }
    }
)

async def run_agent_with_mcp():
    # IMPORTANT: MCP requires async context
    async with client:
        # Get tools from MCP servers
        mcp_tools = await client.get_tools()
        
        # Create agent with MCP tools
        agent = create_agent(
            model="openai:gpt-4o-mini",
            tools=mcp_tools,  # MCP tools are automatically LangChain-compatible
            prompt="You are a helpful assistant with access to external tools.",
        )
        
        # Invoke agent - it can now use MCP tools
        result = await agent.ainvoke({
            "messages": [{
                "role": "user",
                "content": "What is the current date and time?"
            }]
        })
        
        print(result["messages"][-1].content)
        # Expected: Agent uses time server's get_current_time tool
        # Returns: "The current date and time is..."

# Run the async function
import asyncio
asyncio.run(run_agent_with_mcp())
"""

print("MCP + Agent Integration Pattern:")
print(agent_integration_example)

### Important Notes on Async Patterns

MCP requires async/await patterns because:
1. **MCP servers run as separate processes** - Communication is inherently async
2. **Multiple servers** - Need to coordinate multiple async connections
3. **Better performance** - Non-blocking I/O for tool calls

**Pattern to follow:**
```python
async with client:  # Establishes connections to all MCP servers
    tools = await client.get_tools()  # Async operation
    result = await agent.ainvoke(...)  # Use async invoke
```

## Example 4: Combining MCP Tools with Custom Tools

You can mix MCP tools with your own custom LangChain tools.

In [ ]:
# CONCEPTUAL EXAMPLE - Mixing MCP and custom tools

mixed_tools_example = """
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

# Define custom tools with @tool decorator
@tool
def calculate_tip(bill_amount: float, tip_percentage: float) -> float:
    \"\"\"Calculate tip amount for a restaurant bill.\"\"\"
    return bill_amount * (tip_percentage / 100)

@tool
def convert_temperature(celsius: float) -> dict:
    \"\"\"Convert Celsius to Fahrenheit and Kelvin.\"\"\"
    return {
        "fahrenheit": (celsius * 9/5) + 32,
        "kelvin": celsius + 273.15
    }

custom_tools = [calculate_tip, convert_temperature]

# Configure MCP client
client = MultiServerMCPClient(
    servers={
        "filesystem": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-filesystem", "/tmp"],
        }
    }
)

async def run_agent_with_mixed_tools():
    async with client:
        # Get MCP tools
        mcp_tools = await client.get_tools()
        
        # Combine MCP tools with custom tools
        all_tools = mcp_tools + custom_tools
        
        print(f"Total tools available: {len(all_tools)}")
        print(f"- MCP tools: {len(mcp_tools)}")
        print(f"- Custom tools: {len(custom_tools)}")
        
        # Create agent with all tools
        agent = create_agent(
            model="openai:gpt-4o-mini",
            tools=all_tools,
            prompt="You have access to filesystem operations and utility calculations.",
        )
        
        # Agent can now use both MCP and custom tools
        result = await agent.ainvoke({
            "messages": [{
                "role": "user",
                "content": "Create a file at /tmp/test.txt with the text 'Hello', then calculate a 20% tip on a $50 bill."
            }]
        })
        
        print(result["messages"][-1].content)
        # Agent will:
        # 1. Use MCP filesystem tool to create file
        # 2. Use custom calculate_tip tool for the calculation

import asyncio
asyncio.run(run_agent_with_mixed_tools())
"""

print("Mixed Tools Pattern:")
print(mixed_tools_example)

### Tool Selection Behavior

When you provide multiple tools to an agent:
1. **LLM decides** which tool to use based on tool descriptions
2. **Clear descriptions are crucial** - Make tool descriptions specific and distinct
3. **No conflicts** - MCP and custom tools work seamlessly together
4. **Tool chaining** - Agent can use multiple tools in sequence to complete a task

## Working Example: Custom Tools as MCP Alternatives

Since MCP requires specific server setup, let's create a practical example using custom tools that simulate common MCP server functionality.

In [ ]:
from langchain_core.tools import tool
from datetime import datetime
import json

@tool
def get_current_time() -> str:
    """Get the current date and time in ISO format."""
    return datetime.now().isoformat()

@tool
def read_file(path: str) -> str:
    """Read contents of a file. Returns the file contents as a string."""
    try:
        with open(path, 'r') as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file. Creates the file if it doesn't exist."""
    try:
        with open(path, 'w') as f:
            f.write(content)
        return f"Successfully wrote to {path}"
    except Exception as e:
        return f"Error writing file: {str(e)}"

@tool
def list_directory(path: str) -> str:
    """List files and directories in the specified path."""
    import os
    try:
        items = os.listdir(path)
        return json.dumps(items, indent=2)
    except Exception as e:
        return f"Error listing directory: {str(e)}"

# These tools simulate common MCP server capabilities
simulated_mcp_tools = [get_current_time, read_file, write_file, list_directory]

print("Created simulated MCP tools:")
for tool in simulated_mcp_tools:
    print(f"- {tool.name}: {tool.description}")

### Using Simulated MCP Tools with an Agent

In [ ]:
from langchain.agents import create_agent

# Create agent with our simulated MCP tools
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=simulated_mcp_tools,
    prompt="You are a helpful assistant with access to file system and time tools.",
)

# Test the agent
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "What is the current time? Format your response nicely."
    }]
})

print(result["messages"][-1].content)

### Multi-Step Tool Usage

In [ ]:
# Test multi-step operations
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Create a file called /tmp/test_mcp.txt with the current timestamp, then read it back to me."
    }]
})

print(result["messages"][-1].content)

## Available MCP Servers

The MCP ecosystem includes hundreds of pre-built servers. Here are some popular ones:

### Official MCP Servers (by Anthropic)
- **@modelcontextprotocol/server-filesystem** - File system operations
- **@modelcontextprotocol/server-time** - Time and date utilities
- **@modelcontextprotocol/server-memory** - Key-value memory storage

### Community MCP Servers
- **GitHub MCP** - GitHub API integration (repos, issues, PRs)
- **Slack MCP** - Send messages, read channels
- **PostgreSQL MCP** - Database queries and operations
- **Google Drive MCP** - File access and management
- **Notion MCP** - Access Notion workspaces
- **Puppeteer MCP** - Browser automation
- **Git MCP** - Git operations

### Finding MCP Servers

1. **Official Registry**: https://github.com/modelcontextprotocol
2. **npm Search**: Search for "@modelcontextprotocol" or "mcp-server"
3. **GitHub Topics**: Look for repositories tagged with "model-context-protocol"

### Installing MCP Servers

Most MCP servers are Node.js packages installed via npm/npx:

```bash
# No installation needed - use npx to run directly
npx -y @modelcontextprotocol/server-time

# Or install globally
npm install -g @modelcontextprotocol/server-filesystem
```

Some are Python-based:
```bash
pip install mcp-server-git
```

## Real-World MCP Setup Example

Here's a complete example of how you would set up MCP in a production environment:

In [ ]:
production_setup = """
# production_agent.py
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "langchain>=1.0.0",
#     "langchain-openai",
#     "langchain-mcp-adapters>=0.1.0",
#     "langgraph>=1.0.0",
# ]
# ///

import asyncio
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.tools import tool

# Custom business logic tools
@tool
def calculate_discount(price: float, discount_percent: float) -> float:
    \"\"\"Calculate final price after discount.\"\"\"
    return price * (1 - discount_percent / 100)

# Configure MCP servers
mcp_client = MultiServerMCPClient(
    servers={
        "github": {
            "transport": "stdio",
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-github",
            ],
            "env": {
                "GITHUB_TOKEN": "your-github-token"
            }
        },
        "filesystem": {
            "transport": "stdio",
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "/workspace"
            ],
        },
    }
)

async def main():
    async with mcp_client:
        # Load MCP tools
        mcp_tools = await mcp_client.get_tools()
        
        # Combine with custom tools
        all_tools = mcp_tools + [calculate_discount]
        
        # Create production agent
        agent = create_agent(
            model="openai:gpt-4o",
            tools=all_tools,
            prompt=\"\"\"You are a development assistant with access to:
            - GitHub operations (create issues, PRs, etc.)
            - File system operations
            - Business calculations
            
            Always be helpful and precise in your operations.\"\"\",
        )
        
        # Example usage
        result = await agent.ainvoke({
            "messages": [{
                "role": "user",
                "content": "Create a GitHub issue in my repo 'myproject' titled 'Add MCP integration' with a detailed description."
            }]
        })
        
        print(result["messages"][-1].content)

if __name__ == "__main__":
    asyncio.run(main())
"""

print("Production Setup Example:")
print(production_setup)

## Best Practices and Tips

### 1. Error Handling
Always wrap MCP operations in try-except blocks:
```python
async with client:
    try:
        tools = await client.get_tools()
    except Exception as e:
        print(f"Failed to load MCP tools: {e}")
        # Fall back to custom tools only
```

### 2. Environment Variables
Store MCP server credentials in environment variables:
```python
import os
"env": {
    "GITHUB_TOKEN": os.getenv("GITHUB_TOKEN"),
    "API_KEY": os.getenv("SERVICE_API_KEY"),
}
```

### 3. Development vs Production
- **Development**: Use simulated tools or custom tools for faster iteration
- **Production**: Use MCP servers for standardized, production-ready integrations

### 4. Tool Description Quality
When mixing MCP and custom tools, ensure descriptions are clear and specific:
```python
@tool
def custom_search(query: str) -> str:
    \"\"\"Search our internal knowledge base (NOT web search).\"\"\"
    # Clear distinction from MCP search tools
```

### 5. Monitoring and Logging
Enable LangSmith tracing to monitor MCP tool usage:
```python
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "mcp-production"
```

## Summary

### Key Takeaways

1. **MCP is the future of tool integration** - Standardized, reusable, ecosystem-driven
2. **Works seamlessly with LangChain** - MCP tools automatically become LangChain tools
3. **Requires async patterns** - Always use `async with` and `await`
4. **Mix with custom tools** - Combine MCP servers with project-specific tools
5. **Growing ecosystem** - Hundreds of servers available, backed by major AI companies

### When to Use What

| Scenario | Recommendation |
|----------|---------------|
| Production app with external APIs | Use MCP servers |
| Quick prototype | Use custom tools |
| Reusable tool across projects | Create/use MCP server |
| Business-specific logic | Use custom tools |
| Standard operations (GitHub, DB, etc.) | Use MCP servers |

### Next Steps

1. **Explore MCP servers**: Browse https://github.com/modelcontextprotocol
2. **Try locally**: Install Node.js and test MCP servers on your machine
3. **Build an MCP server**: Create a custom server for your unique needs
4. **Join the community**: Follow MCP developments and contribute

### Resources

- **LangChain MCP Adapters**: https://github.com/langchain-ai/langchain-mcp-adapters
- **MCP Documentation**: https://docs.langchain.com/oss/python/langchain/mcp
- **MCP Specification**: https://github.com/modelcontextprotocol/specification
- **Community Servers**: https://github.com/topics/model-context-protocol

---

**Note**: This notebook focused on concepts and patterns due to MCP's subprocess requirements. For hands-on experience, run these examples in a local Python environment with Node.js installed.